In [9]:
"""
Static Search Strategy Function
"""

from typing import Dict, Any
import logging

logger = logging.getLogger(__name__)


def determine_okdb_search_strategy(vr_data: Dict[str, Any]) -> Dict[str, Any]:
    """
    Static function to determine OK DB search strategy based on entity type
    
    Args:
        vr_data: Parsed VR record containing entity type and identifiers (exact VR API field names)
    
    Returns:
        Dictionary with search parameters and strategy
    """
    try:
        # Validate input data
        if not vr_data or not isinstance(vr_data, dict):
            raise ValueError("VR data must be a non-empty dictionary")
        
        # Extract validation fields using exact VR API field names
        entity_type = vr_data.get("validation.entityTypeIco", "").strip()
        vr_id = vr_data.get("validation.id")
        
        if not entity_type:
            raise ValueError("Entity type is required but missing")
        
        # Field name validation for critical fields
        required_validation_fields = ["validation.entityTypeIco", "validation.id"]
        missing_fields = [field for field in required_validation_fields if field not in vr_data]
        if missing_fields:
            raise ValueError(f"Missing required validation fields: {missing_fields}")
        
        logger.info(f"Determining search strategy for VR ID: {vr_id}, Entity Type: {entity_type}")
        
        # ENT_ACTIVITY Logic
        if entity_type == "ENT_ACTIVITY":
            # Check for individual.originKeyEid first
            individual_origin_key = vr_data.get("individual.originKeyEid", "").strip()
            
            if individual_origin_key:
                logger.debug(f"VR ID {vr_id}: Using originKeyEid search for individual")
                return {
                    "search_method": "originkey_individual",
                    "search_params": {
                        "originKeyEid": individual_origin_key,
                        "searchType": "individual_by_originkey"
                    },
                    "entity_type": entity_type
                }
            
            # Fallback to name-based search
            first_name = vr_data.get("individual.firstName", "").strip()
            last_name = vr_data.get("individual.lastName", "").strip()
            
            if not first_name and not last_name:
                # Field validation for individual data
                individual_fields = ["individual.originKeyEid", "individual.firstName", "individual.lastName"]
                available_fields = [field for field in individual_fields if vr_data.get(field, "").strip()]
                if not available_fields:
                    raise ValueError("Individual must have either originKeyEid or name information")
                raise ValueError("Individual must have either originKeyEid or name information")
            
            search_params = {"searchType": "individual_by_name"}
            
            # Add available fields
            if first_name:
                search_params["firstName"] = first_name
            if last_name:
                search_params["lastName"] = last_name
            
            workplace_name = vr_data.get("workplace.usualName", "").strip()
            if workplace_name:
                search_params["workplace"] = workplace_name
            
            country = vr_data.get("address.country", "").strip()
            if country:
                search_params["country"] = country
            
            city = vr_data.get("address.city", "").strip()
            if city:
                search_params["city"] = city
            
            # Add specialty codes if available
            specialty_code1 = vr_data.get("individual.specialityCode1", "").strip()
            if specialty_code1:
                search_params["specialityCode1"] = specialty_code1
            
            specialty_code2 = vr_data.get("individual.specialityCode2", "").strip()
            if specialty_code2:
                search_params["specialityCode2"] = specialty_code2
                
            specialty_code3 = vr_data.get("individual.specialityCode3", "").strip()
            if specialty_code2:
                search_params["specialityCode3"] = specialty_code3
            
            
            logger.debug(f"VR ID {vr_id}: Using name-based search with {len(search_params)} parameters")
            
            return {
                "search_method": "name_based",
                "search_params": search_params,
                "entity_type": entity_type
            }
        
        # ENT_WORKPLACE Logic
        elif entity_type == "ENT_WORKPLACE":
            # Check for workplace.originKeyEid first
            workplace_origin_key = vr_data.get("workplace.originKeyEid", "").strip()
            
            if workplace_origin_key:
                logger.debug(f"VR ID {vr_id}: Using originKeyEid search for workplace")
                return {
                    "search_method": "originkey_workplace",
                    "search_params": {
                        "originKeyEid": workplace_origin_key,
                        "searchType": "workplace_by_originkey"
                    },
                    "entity_type": entity_type
                }
            
            # Fallback to workplace name-based search
            workplace_name = vr_data.get("workplace.usualName", "").strip()
            
            if not workplace_name:
                # Field validation for workplace data
                workplace_fields = ["workplace.originKeyEid", "workplace.usualName"]
                available_fields = [field for field in workplace_fields if vr_data.get(field, "").strip()]
                if not available_fields:
                    raise ValueError("Workplace must have either originKeyEid or usualName")
                raise ValueError("Workplace must have either originKeyEid or usualName")
            
            search_params = {
                "workplaceName": workplace_name,
                "searchType": "workplace_by_name"
            }
            
            # Add geographical context
            country = vr_data.get("address.country", "").strip()
            if country:
                search_params["country"] = country
            
            city = vr_data.get("address.city", "").strip()
            if city:
                search_params["city"] = city
            
            # Add workplace details
            type_code = vr_data.get("workplace.typeCode", "").strip()
            if type_code:
                search_params["typeCode"] = type_code
            
            activity_location_code = vr_data.get("workplace.activityLocationCode", "").strip()
            if activity_location_code:
                search_params["activityLocationCode"] = activity_location_code
            
            # Add address details
            long_label = vr_data.get("address.longLabel", "").strip()
            if long_label:
                search_params["address"] = long_label
            
            postal_code = vr_data.get("address.longPostalCode", "").strip()
            if postal_code:
                search_params["postalCode"] = postal_code
            
            logger.debug(f"VR ID {vr_id}: Using workplace name-based search")
            
            return {
                "search_method": "workplace_based",
                "search_params": search_params,
                "entity_type": entity_type
            }
        
        # Unknown entity type - fallback
        else:
            logger.warning(f"Unknown entity type: {entity_type}, using fallback strategy")
            
            # Try individual-based search as fallback
            first_name = vr_data.get("individual.firstName", "").strip()
            last_name = vr_data.get("individual.lastName", "").strip()
            
            if first_name or last_name:
                search_params = {"searchType": "fallback_search"}
                
                if first_name:
                    search_params["firstName"] = first_name
                if last_name:
                    search_params["lastName"] = last_name
                
                workplace_name = vr_data.get("workplace.usualName", "").strip()
                if workplace_name:
                    search_params["workplace"] = workplace_name
                
                country = vr_data.get("address.country", "").strip()
                if country:
                    search_params["country"] = country
                
                return {
                    "search_method": "name_based",
                    "search_params": search_params,
                    "entity_type": f"UNKNOWN_{entity_type}"
                }
            
            # Last resort
            raise ValueError("Insufficient data to determine any viable search strategy")
    
    except Exception as e:
        logger.error(f"Error determining search strategy: {str(e)}")
        raise ValueError(f"Failed to determine search strategy: {str(e)}")

In [10]:
# VR Test Data Samples - Using Exact VR API Field Names (with dot notation)

# Sample 1 - ENT_ACTIVITY with individual originKeyEid
vr_sample_1 = {
    "validation.refAreaEid": "RAR_ITALY",
    "validation.id": 1019000131693245,
    "validation.customerId": 2436,
    "validation.externalId": "47064409",
    "validation.customerRequestEid": "VVAIT797937-NA-NA-CGD",
    "validation.vrTypeCode": "VMR",
    "validation.countryCode": "IT",
    "validation.entityTypeIco": "ENT_ACTIVITY",
    "validation.integrationDate": "2025-06-02T18:06:29Z",
    "validation.requestDate": "2025-05-30T12:00:00Z",
    "validation.requestComment": "Claudia Gambini kol426 Presso l'ospedale di Velletri di Via Orti Ginnetti fa due giorni di visite diabetologiche e un giorno di diabete gestazionale-DCR000266334",
    "validation.statusIco": "VAS_NOT_PROCESSED",
    "validation.isForced": False,
    "validation.businessStatusCode": "C",
    "validation.statusDate": "2025-06-02T18:06:29Z",
    "validation.requesterLastName": "dcr_integration_user",
    "validation.slaRemainingNumDays": 1,
    "validation.slaDate": "2025-06-03",
    "validation.slaNumDays": 5,
    "validation.slaWorkInstructionsFileUrl": "http://crm.intranet.imshealth.com/departments/OneKey/DataGovernance/Document_Library/SOPs_Standard_forms_User_guides/WI-ValidationRequestTreatment-IO.pdf",
    "validation.slaWorkInstructionsSummaryCode": "WI_VMR",
    "validation.individualWithoutActivityAccepted": False,
    "individual.originKeyEid": "WITO43596247",
    "individual.firstName": "PAOLO",
    "individual.lastName": "CORVISIERI",
    "individual.specialityCode1": "18",
    "individual.specialityCode2": "29",
    "workplace.originKeyEid": "WITO62731655",
    "workplace.usualName": "DISTRETTO SANITARIO FIUMICINO",
    "address.country": "IT",
    "address.city": "VELLETRI",
    "address.postalCity": "VELLETRI"
}

# Sample 2 - ENT_ACTIVITY without individual originKeyEid (name-based search)
vr_sample_2 = {
    "validation.refAreaEid": "RAR_ITALY",
    "validation.id": 1019000131693115,
    "validation.customerId": 9271,
    "validation.externalId": "47064466",
    "validation.customerRequestEid": "VR44423202506021009443302",
    "validation.vrTypeCode": "VMR",
    "validation.countryCode": "IT",
    "validation.entityTypeIco": "ENT_ACTIVITY",
    "validation.integrationDate": "2025-06-02T12:10:46Z",
    "validation.requestDate": "2025-06-02T12:10:46Z",
    "validation.statusIco": "VAS_NOT_PROCESSED",
    "validation.isForced": False,
    "validation.businessStatusCode": "C",
    "validation.statusDate": "2025-06-02T12:10:46Z",
    "validation.requesterId": "Q053K000000ESbSRQA5",
    "validation.requesterLastName": "Rocco Maiorano",
    "validation.slaRemainingNumDays": 1,
    "validation.slaDate": "2025-06-09",
    "validation.slaNumDays": 5,
    "validation.slaWorkInstructionsFileUrl": "http://crm.intranet.imshealth.com/departments/OneKey/DataGovernance/Document_Library/SOPs_Standard_forms_User_guides/WI-ValidationRequestTreatment-IO.pdf",
    "validation.slaWorkInstructionsSummaryCode": "WI_VMR",
    "validation.individualWithoutActivityAccepted": False,
    # NO individual.originKeyEid - triggers name-based search
    "individual.firstName": "GAETANO",
    "individual.lastName": "CANALE",
    "individual.specialityCode1": "08",
    "individual.specialityCode2": "29",
    "workplace.originKeyEid": "WITE000120355",
    "workplace.usualName": "DISTRETTO SANITARIO FIUMICINO",
    "workplace.typeCode": "DST",
    "workplace.activityLocationCode": "A",
    "workplace.managerUsualName": "ASL ROMA 3",
    "workplace.telephone": "06-56483822",
    "address.country": "IT",
    "address.longLabel": "via coni zugna 173",
    "address.city": "FIUMICINO",
    "address.longPostalCode": "00054",
    "address.postalCity": "FIUMICINO"
}

# Sample 3 - ENT_WORKPLACE with workplace originKeyEid
vr_sample_3 = {
    "validation.refAreaEid": "RAR_ITALY",
    "validation.id": 1019000131743075,
    "validation.customerId": 6675,
    "validation.externalId": "47108375",
    "validation.customerRequestEid": "VR14647202506041502203044",
    "validation.vrTypeCode": "VMR",
    "validation.countryCode": "IT",
    "validation.entityTypeIco": "ENT_WORKPLACE",
    "validation.integrationDate": "2025-06-04T17:05:54Z",
    "validation.requestDate": "2025-06-04T17:05:54Z",
    "validation.statusIco": "VAS_NOT_PROCESSED",
    "validation.isForced": False,
    "validation.businessStatusCode": "C",
    "validation.statusDate": "2025-06-04T17:05:54Z",
    "validation.requesterId": "0055000002NtDyTAK",
    "validation.requesterLastName": "Cristian Lucio Gautiero",
    "validation.slaRemainingNumDays": 3,
    "validation.slaDate": "2025-06-11",
    "validation.slaNumDays": 5,
    "validation.slaWorkInstructionsFileUrl": "http://crm.intranet.imshealth.com/departments/OneKey/DataGovernance/Document_Library/SOPs_Standard_forms_User_guides/WI-ValidationRequestTreatment-IO.pdf",
    "validation.slaWorkInstructionsSummaryCode": "WI_VMR",
    "validation.individualWithoutActivityAccepted": False,
    "workplace.originKeyEid": "WITE09992099",
    "workplace.usualName": "SANITARIA ORTOPEDIA SANTALI",
    "workplace.typeCode": "SAN",
    "workplace.activityLocationCode": "T",
    "address.country": "IT",
    "address.longLabel": "VIA MORIGGIA PELASCINI 3",
    "address.city": "GRAVEDONA ED UNITI",
    "address.longPostalCode": "22015",
    "address.postalCity": "GRAVEDONA ED UNITI"
}

# Sample 4 - ENT_WORKPLACE without originKeyEid (workplace name-based search)
vr_sample_4 = {
    "validation.refAreaEid": "RAR_ITALY", 
    "validation.id": 1019000131693050,
    "validation.customerId": 9102,
    "validation.externalId": "47064455",
    "validation.customerRequestEid": "VR09920250602085629202",
    "validation.vrTypeCode": "VMR",
    "validation.countryCode": "IT",
    "validation.entityTypeIco": "ENT_WORKPLACE",
    "validation.integrationDate": "2025-06-02T10:57:42Z",
    "validation.requestDate": "2025-06-02T10:57:42Z", 
    "validation.statusIco": "VAS_NOT_PROCESSED",
    "validation.isForced": False,
    "validation.businessStatusCode": "C",
    "validation.statusDate": "2025-06-02T10:57:42Z",
    "validation.requesterId": "005KA0000000xmvYAA",
    "validation.requesterLastName": "Vita Fedele",
    "validation.slaRemainingNumDays": 1,
    "validation.slaDate": "2025-06-09",
    "validation.slaNumDays": 5,
    "validation.slaWorkInstructionsFileUrl": "http://crm.intranet.imshealth.com/departments/OneKey/DataGovernance/Document_Library/SOPs_Standard_forms_User_guides/WI-ValidationRequestTreatment-IO.pdf",
    "validation.slaWorkInstructionsSummaryCode": "WI_VMR",
    "validation.individualWithoutActivityAccepted": False,
    # NO workplace.originKeyEid - triggers workplace name-based search
    "workplace.usualName": "Marco Oradei",
    "workplace.specialityCode1": "33",
    "address.country": "IT",
    "address.longLabel": "Largo F. Vito 1, 00168 Roma",
    "address.city": "Roma",
    "address.longPostalCode": "00054",
    "address.postalCity": "Roma"
}

# # Test cases for validation
# test_cases = [
#     {
#         "name": "ENT_ACTIVITY with individual originKeyEid",
#         "data": vr_sample_1,
#         "expected_method": "originkey_individual",
#         "expected_search_type": "individual_by_originkey"
#     },
#     {
#         "name": "ENT_ACTIVITY without individual originKeyEid", 
#         "data": vr_sample_2,
#         "expected_method": "name_based",
#         "expected_search_type": "individual_by_name"
#     },
#     {
#         "name": "ENT_WORKPLACE with workplace originKeyEid",
#         "data": vr_sample_3,
#         "expected_method": "originkey_workplace",
#         "expected_search_type": "workplace_by_originkey"
#     },
#     {
#         "name": "ENT_WORKPLACE without workplace originKeyEid",
#         "data": vr_sample_4,
#         "expected_method": "workplace_based",
#         "expected_search_type": "workplace_by_name"
#     }
# ]

# # Test function
# def test_search_strategy():
#     """Test the search strategy function with exact VR API field names"""
#     from utils.search_strategy import determine_okdb_search_strategy
    
#     print("=== Testing VR Search Strategy Function with Exact API Field Names ===\n")
    
#     for i, test_case in enumerate(test_cases, 1):
#         print(f"Test {i}: {test_case['name']}")
#         print(f"VR ID: {test_case['data']['validation.id']}")
#         print(f"Entity Type: {test_case['data']['validation.entityTypeIco']}")
        
#         try:
#             result = determine_okdb_search_strategy(test_case['data'])
            
#             print(f"✅ SUCCESS")
#             print(f"   Search Method: {result['search_method']}")
#             print(f"   Search Type: {result['search_params']['searchType']}")
#             print(f"   Entity Type: {result['entity_type']}")
#             print(f"   Parameters: {len(result['search_params'])} total")
            
#             # Validate expected results
#             if result['search_method'] == test_case['expected_method']:
#                 print(f"   ✅ Method matches expected: {test_case['expected_method']}")
#             else:
#                 print(f"   ❌ Method mismatch! Expected: {test_case['expected_method']}, Got: {result['search_method']}")
            
#             if result['search_params']['searchType'] == test_case['expected_search_type']:
#                 print(f"   ✅ Search type matches expected: {test_case['expected_search_type']}")
#             else:
#                 print(f"   ❌ Search type mismatch! Expected: {test_case['expected_search_type']}, Got: {result['search_params']['searchType']}")
            
#             # Show key parameters
#             key_params = {k: v for k, v in result['search_params'].items() if k != 'searchType'}
#             if key_params:
#                 print(f"   Key Parameters: {key_params}")
                
#         except Exception as e:
#             print(f"❌ ERROR: {str(e)}")
        
#         print("-" * 60)
    
#     print("=== Test Complete ===")

# if __name__ == "__main__":
#     test_search_strategy()

In [11]:
determine_okdb_search_strategy(vr_sample_1)

{'search_method': 'originkey_individual',
 'search_params': {'originKeyEid': 'WITO43596247',
  'searchType': 'individual_by_originkey'},
 'entity_type': 'ENT_ACTIVITY'}

In [12]:
determine_okdb_search_strategy(vr_sample_2)

{'search_method': 'name_based',
 'search_params': {'searchType': 'individual_by_name',
  'firstName': 'GAETANO',
  'lastName': 'CANALE',
  'workplace': 'DISTRETTO SANITARIO FIUMICINO',
  'country': 'IT',
  'city': 'FIUMICINO',
  'specialityCode1': '08',
  'specialityCode2': '29',
  'specialityCode3': ''},
 'entity_type': 'ENT_ACTIVITY'}

In [13]:
determine_okdb_search_strategy(vr_sample_3)

{'search_method': 'originkey_workplace',
 'search_params': {'originKeyEid': 'WITE09992099',
  'searchType': 'workplace_by_originkey'},
 'entity_type': 'ENT_WORKPLACE'}

In [14]:
determine_okdb_search_strategy(vr_sample_4)

{'search_method': 'workplace_based',
 'search_params': {'workplaceName': 'Marco Oradei',
  'searchType': 'workplace_by_name',
  'country': 'IT',
  'city': 'Roma',
  'address': 'Largo F. Vito 1, 00168 Roma',
  'postalCode': '00054'},
 'entity_type': 'ENT_WORKPLACE'}